# Query Transformations for Enhanced RAG Systems

이 노트북에서는 LangChain 같은 특수 라이브러리 없이도 RAG 시스템의 검색 성능을 높일 수 있는 세 가지 쿼리 변환 기법을 소개합니다. 사용자의 질문을 조금만 바꿔도, 더 관련성 높고 풍부한 정보를 쉽게 찾을 수 있습니다.
 
#### 주요 쿼리 변환 방법

1. **Query Rewriting**: 질문을 더 구체적이고 자세하게 바꿔서 검색 정확도를 높입니다.
2. **Step-back Prompting**: 질문을 더 넓은 범위로 확장해, 배경이나 맥락 정보를 함께 찾을 수 있게 합니다.
3. **Sub-query Decomposition**: 복잡한 질문을 여러 개의 간단한 질문으로 나눠서, 더 폭넓고 꼼꼼하게 정보를 검색합니다.

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

# OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## Query Transformation 구현하기
### 1. Query Rewriting

`Query Rewriting`은 사용자의 질문을 구체적이고 자세하게 바꿔서, 원하는 정보를 확하게 찾을 수 있게 해줍니다.

In [ ]:
def rewrite_query(original_query, model_name="gpt-4.1-mini"):
    """
    사용자의 질문을 구체적이고 자세하게 변환.
    
    Args:
        original_query (str): 원본 사용자 쿼리
        model_name (str): 쿼리 리라이팅에 사용할 모델
        
    Returns:
        str: 리라이팅된 쿼리
    """
    # 시스템 프롬프트
    system_prompt = "You are an AI assistant specialized in improving search queries. Your task is to rewrite user queries to be more specific, detailed, and likely to retrieve relevant information in korea."
    
    # 사용자 프롬프트
    user_prompt = f"""
    Rewrite the following query to make it more specific and detailed. Include relevant terms and concepts that might help in retrieving accurate information.
    
    Original query: {original_query}
    
    Rewritten query:
    """
    
    # 지정된 모델을 사용하여 Query Rewriting
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0.0,  # temperature는 매우 낮게 설정
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 리라이팅된 쿼리를 반환하고, 앞뒤 공백을 제거합니다.
    return response.choices[0].message.content.strip()

### 2. Step-back Prompting
더 넓은 범위의 쿼리를 만들어서 배경 정보나 맥락을 파악하는 데 도움을 줍니다.

In [ ]:
def generate_step_back_query(original_query, model_name="gpt-4.1-mini"):
    """
    더 넓은 범위의 쿼리 생성
    
    Args:
        original_query (str): 원본 사용자 쿼리
        model_name (str): 스텝백 쿼리 생성에 사용할 모델
        
    Returns:
        str: 스텝백 쿼리
    """
    # 시스템 프롬프트
    system_prompt = "You are an AI assistant specialized in search strategies. Your task is to generate broader in korea, more general versions of specific queries to retrieve relevant background information."
    
    # 사용자 프롬프트
    user_prompt = f"""
    Generate a broader, more general version of the following query that could help retrieve useful background information.
    
    Original query: {original_query}
    
    Step-back query:
    """
    
    # 지정된 모델을 사용하여 step-back query 생성
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0.1,  
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 스텝백 쿼리를 return하고, 앞뒤 공백 제거
    return response.choices[0].message.content.strip()

### 3. Sub-query Decomposition
복잡한 쿼리를 더 단순한 질문들로 나누어, 다양한 측면에서 정보를 폭넓게 찾을 수 있도록 도와줍니다.

In [ ]:
def decompose_query(original_query, num_subqueries=4, model_name="gpt-4.1-mini"):
    """
    복잡한 쿼리를 더 간단한 서브쿼리로 분해합니다.
    
    Args:
        original_query (str): 원본 복잡한 쿼리
        num_subqueries (int): 생성할 서브쿼리의 수
        model_name (str): 쿼리 분해에 사용할 모델
        
    Returns:
        List[str]: 더 간단한 서브쿼리 리스트
    """
    # 시스템 프롬프트
    system_prompt = "You are an AI assistant specialized in breaking down complex questions. Your task is to decompose complex queries into simpler sub-questions that, when answered together, address the original query in korea"
    
    # 사용자 프롬프트
    user_prompt = f"""
    Break down the following complex query into {num_subqueries} simpler sub-queries. Each sub-query should focus on a different aspect of the original question.
    
    Original query: {original_query}
    
    Generate {num_subqueries} sub-queries, one per line, in this format:
    1. [First sub-query]
    2. [Second sub-query]
    And so on...
    """
    
    # 지정된 모델을 사용하여 서브쿼리를 생성합니다.
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0.2,  
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 응답에서 서브쿼리 추출
    content = response.choices[0].message.content.strip()
    
    # 번호가 붙은 쿼리 추출
    lines = content.split("\n")
    sub_queries = []
    
    for line in lines:
        if line.strip() and any(line.strip().startswith(f"{i}.") for i in range(1, 10)):
            query = line.strip()
            query = query[query.find(".")+1:].strip()
            sub_queries.append(query)
    
    return sub_queries

## 쿼리 변환 기법 비교
다양한 쿼리 변환 기법을 예시 쿼리에 적용해 보겠습니다.

In [ ]:
# 예시 쿼리
original_query = "AI가 미래에 어떤 직업들에게 영향을 미칠까요?"

# 쿼리 변환 적용
print("Original Query:", original_query)

# Query Rewriting
rewritten_query = rewrite_query(original_query)
print("\n1. Rewritten Query:")
print(rewritten_query)

# Step-back Prompting
step_back_query = generate_step_back_query(original_query)
print("\n2. Step-back Query:")
print(step_back_query)

# Sub-query Decomposition
sub_queries = decompose_query(original_query, num_subqueries=4)
print("\n3. Sub-queries:")
for i, query in enumerate(sub_queries, 1):
    print(f"   {i}. {query}")

## Simple Vector Store 구축
NumPy를 사용하여 간단한 Vecotr store 구축

In [ ]:
class SimpleVectorStore:
    """
    NumPy를 사용하여 간단한 Vecotr store 구축
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목 추가

        Args:
        text (str): 원본 텍스트.
        embedding (List[float]): 임베딩 벡터.
        metadata (dict, optional): 추가 메타데이터.
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def similarity_search(self, query_embedding, k=5):
        """
        시맨틱 서치 수행

        Args:
        query_embedding (List[float]): 쿼리 임베딩 벡터.
        k (int): 반환할 결과의 수.

        Returns:
        List[Dict]: 텍스트와 메타데이터가 포함된 상위 k개 유사 항목.
        """
        if not self.vectors:
            return []
        
        # 쿼리 임베딩을 numpy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 사용하여 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도에 따라 정렬 (내림차순)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })
        
        return results

## 임베딩 생성

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

## 쿼리 변환을 적용한 RAG 구현

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

In [ ]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로, 일부가 겹치도록 chunking 합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 리스트
    
    # overlap만큼 겹치도록 text를 n의 길이로 chunking
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])

    return chunks  

In [ ]:
def process_document(file_path, chunk_size=1000, chunk_overlap=200):
    """
    질문 생성을 통한 Document Augmentation 

    Args:
    pdf_path (str): PDF 파일 경로.
    chunk_size (int): 청크의 길이.
    chunk_overlap (int): 청크 간 겹치는 문자 수.

    Returns:
    Tuple[List[str], SimpleVectorStore]: 텍스트 청크와 벡터 저장소.
    """
    # print("Extracting text from PDF...")
    # extracted_text = extract_text_from_pdf(file_path)

    # 텍스트 파일 로드
    extracted_text = load_text_file(file_path)
    
    print("Chunking text...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"Created {len(chunks)} text chunks")
    
    # 텍스트 청크의 임베딩 생성
    print("Creating embeddings for chunks...")
    chunk_embeddings = create_embeddings(embedding_model, chunks, device=device, batch_size=1)
    
    # 벡터 저장소 생성
    store = SimpleVectorStore()
    
    # 각 청크와 임베딩을 VectorStore에 저장
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": file_path}
        )
    
    print(f"Added {len(chunks)} chunks to the vector store")
    return store

## RAG with Query Transformations

In [ ]:
def transformed_search(query, vector_store, transformation_type, top_k=3):
    """
    변환된 쿼리를 사용하여 유사 텍스트(청크) 검색
    
    Args:
        query (str): 원본 쿼리
        vector_store (SimpleVectorStore): 검색할 벡터 저장소
        transformation_type (str): 변환 유형 ('rewrite', 'step_back', or 'decompose')
        top_k (int): 반환할 결과의 수
        
    Returns:
        List[Dict]: 검색 결과
    """
    print(f"Transformation type: {transformation_type}")
    print(f"Original query: {query}")
    
    results = []
    
    if transformation_type == "rewrite":
        # Query rewriting
        transformed_query = rewrite_query(query)
        print(f"Rewritten query: {transformed_query}")
        
        # 변환된 쿼리의 임베딩 생성
        query_embedding = create_embeddings(embedding_model, [transformed_query], device=device, batch_size=1)[0]
        
        # rewriting된 쿼리로 검색
        results = vector_store.similarity_search(query_embedding, k=top_k)
        
    elif transformation_type == "step_back":
        # 스텝백 프롬프트
        transformed_query = generate_step_back_query(query)
        print(f"Step-back query: {transformed_query}")
        
        # 변환된 쿼리의 임베딩 생성
        query_embedding = create_embeddings(embedding_model, [transformed_query], device=device, batch_size=1)[0]
        
        # 스텝백 쿼리로 검색
        results = vector_store.similarity_search(query_embedding, k=top_k)
        
    elif transformation_type == "decompose":
        # 서브쿼리 분해
        sub_queries = decompose_query(query)
        print("Decomposed into sub-queries:")
        for i, sub_q in enumerate(sub_queries, 1):
            print(f"{i}. {sub_q}")
        
        # 모든 서브쿼리의 임베딩 생성
        sub_query_embeddings = create_embeddings(embedding_model, sub_queries, device=device, batch_size=1)
        
        # 각 서브쿼리로 검색하고 결과 결합
        all_results = []
        for i, embedding in enumerate(sub_query_embeddings):
            sub_results = vector_store.similarity_search(embedding, k=2)  # 각 서브쿼리당 결과 수 줄임
            all_results.extend(sub_results)
        
        # 중복 제거 (가장 높은 유사도 점수 유지)
        seen_texts = {}
        for result in all_results:
            text = result["text"]
            if text not in seen_texts or result["similarity"] > seen_texts[text]["similarity"]:
                seen_texts[text] = result
        
        # 유사도 순으로 정렬하고 top_k 결과 선택
        results = sorted(seen_texts.values(), key=lambda x: x["similarity"], reverse=True)[:top_k]
        
    else:
        # 변환 없이 일반 검색
        query_embedding = create_embeddings(query)
        results = vector_store.similarity_search(query_embedding, k=top_k)
    
    return results

## 검색된 청크를 기반으로 response 생성하기

In [ ]:
def generate_response(query, context, model_name='gpt-4.1-nano'):

    # 시스템 프롬프트
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
    
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        Please answer the question based only on the context provided above. Be concise and accurate.
    """
    
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content

## Running the Complete RAG Pipeline with Query Transformations

In [ ]:
def rag_with_query_transformation(file_path, query, transformation_type=None):
    """
    쿼리 변환을 선택적으로 적용하여 전체 RAG 파이프라인 실행
    
    Args:
        file_path (str): 파일 경로
        query (str): 사용자 쿼리
        transformation_type (str): 변환 유형 (None, 'rewrite', 'step_back', or 'decompose')
        
    Returns:
        Dict: 쿼리, 변환된 쿼리, 컨텍스트, 응답이 포함된 결과
    """
    # 벡터 저장소 생성
    vector_store = process_document(file_path)
    
    # 쿼리 변환 적용 및 검색
    if transformation_type:
        # 변환된 쿼리로 검색
        results = transformed_search(query, vector_store, transformation_type)
    else:
        # 변환 없이 원본 쿼리로 검색
        query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)[0]
        results = vector_store.similarity_search(query_embedding, k=3)
    
    # 검색 결과에서 컨텍스트 결합
    context = "\n\n".join([f"PASSAGE {i+1}:\n{result['text']}" for i, result in enumerate(results)])
    
    # 쿼리와 결합된 컨텍스트를 기반으로 응답 생성
    response = generate_response(query, context)
    
    # 원본 쿼리, 변환 유형, 컨텍스트, 응답이 포함된 결과 반환
    return {
        "original_query": query,
        "transformation_type": transformation_type,
        "context": context,
        "response": response
    }

## Evaluating Transformation Techniques

In [ ]:
def compare_responses(results, reference_answer, model_name="gpt-4.1-mini"):
    """
    다른 쿼리 변환 기법의 응답을 비교합니다.
    
    Args:
        results (Dict): 다른 변환 기법의 결과
        reference_answer (str): 비교를 위한 기준 답변
        model (str): 평가에 사용할 모델
    """
    # 시스템 프롬프트
    system_prompt = """You are an expert evaluator of RAG systems. 
    Your task is to compare different responses generated using various query transformation techniques 
    and determine which technique produced the best response compared to the reference answer."""
    
    # 기준 답변과 각 기법의 응답을 비교하기 위한 텍스트 준비
    comparison_text = f"""Reference Answer: {reference_answer}\n\n"""
    
    for technique, result in results.items():
        comparison_text += f"{technique.capitalize()} Query Response:\n{result['response']}\n\n"
    
    # 유저 프롬프트
    user_prompt = f"""
    {comparison_text}
    
    Compare the responses generated by different query transformation techniques to the reference answer.
    
    For each technique (original, rewrite, step_back, decompose):
    1. Score the response from 1-10 based on accuracy, completeness, and relevance
    2. Identify strengths and weaknesses
    
    Then rank the techniques from best to worst and explain which technique performed best overall and why.
    """
    
    # 응답 평가
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 평가 결과 출력
    print("\n===== EVALUATION RESULTS =====")
    print(response.choices[0].message.content)
    print("=============================")

In [ ]:
def evaluate_transformations(file_path, query, reference_answer=None):
    """
        다른 쿼리 변환 기법의 응답을 비교합니다.
    
    Args:
        file_path (str): 파일 경로
        query (str): 평가할 쿼리
        reference_answer (str): 비교를 위한 기준 답변
        
    Returns:
        Dict: Evaluation results
    """
    # 평가할 쿼리 변환 기법 정의
    transformation_types = [None, "rewrite", "step_back", "decompose"]
    results = {}
    
    # 각각의 방법들로 RAG pipeline실행
    for transformation_type in transformation_types:
        type_name = transformation_type if transformation_type else "original"
        print(f"\n===== Running RAG with {type_name} query =====")
        
        # 결과 가져오기
        result = rag_with_query_transformation(file_path, query, transformation_type)
        results[type_name] = result
        
        # 응답 출력
        print(f"Response with {type_name} query:")
        print(result["response"])
        print("=" * 50)
    
    # 기준 답변이 제공된 경우 결과 비교
    if reference_answer:
        compare_responses(results, reference_answer)
    
    return results

## 평가

In [ ]:
import pandas as pd

# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')

# 평가 데이터에서 첫 번째 쿼리 추출
query = df['query'][0]

# 참고(정답) 답변
reference_answer = df['generation_gt'][0]

# 파일 경로
file_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

# 응답 평가
evaluation_results = evaluate_transformations(file_path, query, reference_answer)